In [22]:
# import pandas as pd

# # Load the dataset
# df = pd.read_excel('data/Adidas US Sales Datasets.xlsx')

# # Display basic information about the dataset
# print(df.info())

# # Display the first few rows to understand the data
# print(df.head())

# file_path = 'data/Adidas US Sales Datasets.xlsx'

# # Load the Excel file and check the sheet names
# xls = pd.ExcelFile(file_path)
# xls.sheet_names

# # Clean the dataset by skipping the initial empty rows and using the correct headers
# df = pd.read_excel(file_path, sheet_name='Data Sales Adidas', skiprows=4)

# # Display the cleaned data to confirm the structure
# df.head()

# df=df.drop(columns=['Unnamed: 0','Retailer ID'])

# # Save the cleaned dataset to a CSV file
# output_csv_path = 'data/Adidas_US_Sales_Cleaned.csv'
# df.to_csv(output_csv_path, index=False)

# output_csv_path



In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib

# Load the user's uploaded CSV file
file_path = 'data/Adidas_US_Sales_Cleaned.csv'
data = pd.read_csv(file_path)



# Selecting relevant features for the model (dropping non-numeric and categorical data)
# We'll predict 'Total Sales' based on 'Price per Unit', 'Units Sold', and 'Operating Profit'
X = data[['Price per Unit', 'Units Sold', 'Operating Profit']]
y = data['Total Sales']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Making predictions on the test data
y_pred = model.predict(X_test)

# Save the trained model to a file using joblib
model_file_path = 'data/linear_regression_model.pkl'
joblib.dump(model, model_file_path)

# Calculating the model's performance using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Coefficients of the model
coefficients = model.coef_

# Displaying the results
mse, coefficients

(np.float64(1253241097.91393),
 array([991.90177354, 221.33278234,   1.62362108]))

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Step 1: Load the dataset
data = pd.read_csv('data/Adidas_US_Sales_Cleaned.csv')

# Step 2: Convert 'Invoice Date' to datetime and extract year and month
data['Invoice Date'] = pd.to_datetime(data['Invoice Date'])
data['year'] = data['Invoice Date'].dt.year
data['month'] = data['Invoice Date'].dt.month

# Step 3: Drop unnecessary columns (if needed)
data = data.drop(['Invoice Date', 'Operating Profit', 'Operating Margin'], axis=1)

# Step 4: One-hot encoding for categorical variables
data = pd.get_dummies(data, columns=['Retailer', 'Region', 'State', 'City', 'Product', 'Sales Method'], drop_first=True)

# Step 5: Define the features and target
X = data.drop(['Units Sold'], axis=1)  # Features
y = data['Units Sold']  # Target variable

# Step 6: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Scale the features (especially for models that need scaled input)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler and columns
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(X.columns, 'columns.pkl')


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Step 8: Train a linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Step 9: Evaluate the model
y_pred = lr_model.predict(X_test_scaled)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

# Step 10: Save the trained model
joblib.dump(lr_model, 'linear_regression_sales_model.joblib')


import pandas as pd
import joblib

# Load the trained model, scaler, and columns
lr_model = joblib.load('linear_regression_sales_model.joblib')
scaler = joblib.load('scaler.joblib')
original_columns = joblib.load('columns.pkl')

# Define a sample input for prediction (future date)
test_data = {
    'year': [2025],
    'month': [1],
    'Price per Unit': [50],  # Add necessary numerical features
    'Retailer_Foot Locker': [1],  # One-hot encoded values
    'Product_Men\'s Street Footwear': [1],
    # Add other one-hot encoded columns and default them to 0
}

# Convert to DataFrame
input_df = pd.DataFrame(test_data)

# Add missing columns (if any) with default value 0
for col in original_columns:
    if col not in input_df.columns:
        input_df[col] = 0

# Ensure correct column order
input_df = input_df[original_columns]

# Scale the input data
input_scaled = scaler.transform(input_df)

# Predict sales
predicted_qty = lr_model.predict(input_scaled)
print(f"Predicted Quantity Sold for {test_data['year'][0]}-{test_data['month'][0]}: {predicted_qty[0]}")


RMSE: 59.3676250059195
R^2: 0.9253291184405359
Predicted Quantity Sold for 2025-1: -1289664079590267.5


/Users/ismatsamadov/Sales_Forecasting/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/var/folders/pc/6rmkz3b536l1k66dbrhqbl2r0000gn/T/ipykernel_3809/1715116000.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
/var/folders/pc/6rmkz3b536l1k66dbrhqbl2r0000gn/T/ipykernel_3809/1715116000.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame,